# Getting Started
Let's start by printing out the version of Julia we're using:

In [4]:
using InteractiveUtils # avoid deprecation warning
versioninfo()

Julia Version 0.7.0-beta2.0
Commit b145832402* (2018-07-13 19:54 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i5-4258U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, haswell)


Note the `using InteractiveUtils` command. In Julia 1.0 much functionality is being moved out of the `Base` library and into different modules in the standard library. In Julia 0.7 such commands will still work without the appropriate `using` statement but a deprecation warning will be emitted, such as:
```
WARNING: Base.versioninfo is deprecated: it has been moved to the standard library package `InteractiveUtils`.
Add `using InteractiveUtils` to your imports.
  likely near In[1]:1
```

Here the `using` statements are included before the first use of any such commands so that this notebook should be usable in Julia 1.0 when the deprecation warnings become errors.